In [1]:
from urllib2 import urlopen
from urllib import urlencode
import json
from datetime import datetime as dt,tzinfo,timedelta

In [2]:
class UTC(tzinfo):
    """UTC"""

    def utcoffset(self, dt):
        return timedelta(0)

    def tzname(self, dt):
        return "UTC"

    def dst(self, dt):
        return timedelta(0)

utc = UTC()

In [3]:
class ECWConnection():
    def __init__(self,host):
        self.__host__ = host

    #send data to server
    def sendData(self,sensorId,datetime,temp,hum,press):
        params   = urlencode({'temp':temp,'hum':hum,'press':press,'datetime':datetime,'sensorId':sensorId})
        response = urlopen('%s/newentry/insert?%s' % (self.__host__,params))
        return json.load(response)
    
    #prediction: json string of the form {'timestamp':temperature_value, ... }
    def updatePrediction(self,prediction):
        params   = urlencode({'prediction':prediction})
        response = urlopen('%s/newentry/insertPrediction?%s' % (self.__host__,params))
        return json.load(response)
    
    def getLatest(self):
        response = urlopen('%s/getreport/' % self.__host__)
        return json.load(response)

In [4]:
#connection = ECWConnection(host='http://127.0.0.1:8000')#django local
connection = ECWConnection(host=' http://0.0.0.0:5000')#herokulocal
#connection = ECWConnection(host='https://ecollectweb.herokuapp.com')#heroku remote

In [31]:
res = connection.sendData(sensorId=1,datetime=dt.now(tz=utc),temp=34,hum=50,press=1000)

In [32]:
res

{u'status': u'succeeded'}

In [30]:
#send prediction data to server
res = connection.updatePrediction(prediction={"10":61,"11":65,"12":71,"1":75})
res

{u'status': u'succeeded'}

In [33]:
#get data from server
res = connection.getLatest()
res

{u'latest_temp': u'34.00',
 u'prediction': {u'1': 75, u'10': 61, u'11': 65, u'12': 71}}